In [221]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import pandas as np
import re

In [242]:
# Create arrays to store housing information.

listing_ids=[]
lat_long=[]
price=[]
bed=[]
bath=[]
size=[]
location=[]

# Iterate through all of the .html files located in html_files folder.
# HTMLs from https://streeteasy.com/for-rent/manhattan?page=X
for x in [*range(0,100)]:
    soup = BeautifulSoup(open("html_files/" + str(x) + ".html"), "html.parser")
    lst = soup.find_all("article")
    for i in range(len(lst)):
        
        # Check to see if unique listing ID is already in the array.
        listing = int(re.findall(r"(?<=data-gtm-listing-id=\").*?(?=\")",str(lst[i]))[0])
        if listing in listing_ids:
            break
        else:
            listing_ids.append(listing)

        # Extract latitude and longitude coordinates.
        lat_long.append(re.findall(r"(?<=point=\").*?(?=\")",str(lst[i])))
        
        # Extract monthly rent price.
        if lst[i].find_all('span',{'class':'price'})==[]:
            price.append('')
        else:
            price.append(lst[i].find_all('span',{'class':'price'})[0].string.replace('$','').replace(',',''))
         
        # Extract the number of bedrooms, bathrooms, and square footage if available.
        temp_list = []
        temp_beds = 0
        temp_baths = 0
        temp_size = 0

        for j in lst[i].find_all('li',{'class':['first_detail_cell','detail_cell','last_detail_cell']}):
            temp_list.append(j.string)
        temp_list = [*map(str,temp_list)]

        for j in temp_list:
            if 'bed' in j:
                temp_beds = float(re.findall(r'\d+', j)[0])
            if 'bath' in j:
                temp_baths = float(re.findall(r'\d+', j)[0])
            if 'ft' in j:
                temp_size = float(re.findall(r'\d+', j)[0])
        
        bed.append(temp_beds)
        bath.append(temp_baths)
        size.append(temp_size)
        
        # Extract the address and unit number for apartment listing.
        location.append(str(lst[i].find_all('a', {'class':'details-titleLink'})[0].string))
        
print('done')

done


In [243]:
len(listing_ids), len(lat_long), len(price), len(bed), len(bath), len(size), len(location)

(883, 883, 883, 883, 883, 883, 883)

In [245]:
housing_df = pd.DataFrame(data={"Listing ID":listing_ids,"Location":location,"Lat_Long":lat_long, "Monthly Rental Price":price,
                               "Bedrooms":bed,"Bathrooms":bath,"Square Footage":size})
housing_df.head()

,Listing ID,Location,Lat_Long,Monthly Rental Price,Bedrooms,Bathrooms,Square Footage
0,3021099,211 North End Avenue #12A,"[40.71628546,-74.01546103]",4600,1.0,1.0,860.0
1,3020988,150 East 39th Street #601,"[40.74890137,-73.97660065]",2650,0.0,1.0,0.0
2,3026635,257 West 136th Street #4AA,"[40.81700815,-73.94508554]",2195,2.0,1.0,0.0
3,3026633,272 West 139th Street #3A,"[40.81876179,-73.9446072]",2650,2.0,1.0,0.0
4,3026626,408 West 34th Street #6E,"[40.75329123,-73.9971017]",1975,0.0,1.0,0.0


In [246]:
housing_df.to_csv('manhattan_rentals_apr27.csv')